<a href="https://colab.research.google.com/github/stevenkolawole/indaba-low-resource-nlp-prac/blob/main/llm_Incontext_learning_w_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from openai import OpenAI

In [ ]:
import openai
from datasets import load_dataset
from rouge_score import rouge_scorer
from tqdm import tqdm
import numpy as np

In [ ]:
client = OpenAI(api_key = os.environ.get('API_KEY'))

In [ ]:
# Load the dataset
dataset = load_dataset("Goud/Goud-sum", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/139288 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9497 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9497 [00:00<?, ? examples/s]

In [ ]:
# Function to generate summaries
def generate_summary(text, examples, model='gpt-4o'):
    prompt = "Generate headlines for the following Moroccan Darija text articles as shown in the examples:\n\n"

    # Add few-shot examples
    for example in examples:
        prompt += f"Text: {example['article']}\n"
        prompt += f"Headline: {example['headline']}\n\n"

    prompt += f"Text: {text}\nHeadline:"

    response = client.chat.completions.create(model="gpt-4o",
                                              messages=[{"role": "user", "content": prompt}])

    return response.choices[0].message.content.strip()


In [ ]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Select a subset of the dataset for evaluation
eval_subset = dataset.select(range(50))

# Select few-shot examples for in-context learning
few_shot_examples = dataset.select(range(51, 61))

# Generate summaries and calculate ROUGE scores
rouge_scores = []

for item in tqdm(eval_subset):
    generated_summary = generate_summary(item['article'], few_shot_examples, model='gpt-4o-mini')
    print('****')
    print("Actual Article: ", item['article'])
    print("Generated Headline: ", generated_summary)
    print()
    scores = scorer.score(item['headline'], generated_summary)
    rouge_scores.append(scores)

# Calculate average ROUGE scores
avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")

  2%|▏         | 1/50 [00:01<01:07,  1.38s/it]

****
Actual Article:  منير العلمي من مراكش: تحول فضاء مقر الغرفة الفلاحية بمدينة مراكش، الذي يحتضن في هذه الأثناء، انتخاب رئيس وأعضاء المكتب المسير للغرفة الفلاحية بجهة مراكش آسفي، إلى حلبة للاشتباكات والملاسنات، بعد اشتداد الخلاف بين البرلمانيين حميد العكرود وعمر خفيف، اللذين ينتميان إلى حزب التجمع الوطني للأحرار، ما كاد يعصف بالاجتماع بعد انطلاق شرارة الاشتباك بالأيادي التي أجهضت في مهدها بتدخل بعض الحاضرين. وحسب شهود عيان، فإن عمر خفيف، الذي يشغل رئيس جماعة أكفاي، ومدعم الحبيب بن الطالب المنسق الاقليمي لحزب الأصالة والمعاصر الذي يتجه لتولي رئاسة الغرفة لولاية تانية، رفض دخول حميد العكرود للمنافسة على رئاسة الغرفة، واصفا إياه بـ “الأمي الذي لايفقه شيئا”، ليدخل الطرفان في ملاسنات كلامية قبل أن يتحول الصراع إلى تشابك بالأيدي. 
Generated Headline:  اشتباكات وملاسنات بمقر الغرفة الفلاحية بمراكش بين برلمانيين من حزب التجمع الوطني للأحرار



  4%|▍         | 2/50 [00:02<01:09,  1.45s/it]

****
Actual Article:  لا حديث في الجسم القضائي الوطني سوى عن الحكم القضائي الذي أصدرته المحكمة الابتدائية بالرباط، في حق الشركة الوطنية للنقل واللوجيستيك، التابعة لوزارة التجهيز والنقل واللوجيستيك والما، والقاضي بأداء هذه الأخيرة لشركة “صوناصر” للبناء، الكائن مقرها الاجتماعي بالمنطقة الصناعية سيدي بوزكري بمكناس، تعويضا ماليا يفوق 13 مليار سنتيم جراء الأضرار التي تسببت لها في الدعاوى القضائية التي رفعتها ضدها، والتي أدت إلى أحكام بالحجز على شاحنتها، معداتها، الاتها ووسائل اشتغالها، فضلا عن جميع حساباتها البنكية وأيضا ديونها لدى الغير. وحسب مصادر موثوقة، فإن قاضي الحكم بابتدائية الرباط، اعتمد في قراره كليا، على تقرير الخبرة المنجزة حول القضية. تفاصيل أخرى في عدد يوم غد. 
Generated Headline:  الشركة الوطنية للنقل واللوجستيك محكومة بأداء تعويض يفوق 13 مليار سنتيم لشركة "صوناصر"



  6%|▌         | 3/50 [00:04<01:18,  1.67s/it]

...

****
Actual Article:  طالب حزب العدالة والتنمية بتوسيع نمط الاقتراع باللائحة وفق أكبر بقية واعتماد عتبة 6 % في الاتتخابات الجماعية. وقال البي جي دي في مذكرته حول اصلاح القوانين الانتخابية إن “حجم المشاركة في القرى هو أعلى منه في المدن. وفي هذا الإطار نقترح تعميم نظام الاقتراع باللائحة على الجماعات التي يتجاوز عدد سكانها 20000 نسمة”. وبخصوص العتبة الانتخابية، قالت المذكرة :”ومن أجل تعزيز ترشيد وعقلنة الخريطة السياسية، نقترح اعتماد عتبة 6 % في الدوائر المحلية التشريعية، والانتخابات الجهوية والجماعية، والاحتفاظ بعتبة 3 % بالنسبة لدائرة اللائحة الوطنية”. 
Generated Headline:  البيجيدي يطالب برفع العتبة إلى 6% وتوسيع نظام الاقتراع باللائحة في الجماعات الكبرى



 98%|█████████▊| 49/50 [01:16<00:01,  1.33s/it]

****
Actual Article:  في أول رد لها على اتهامها من قبل الحكومة بـ”تحريض” طلبة الطب على الاحتجاج، اعتبرت جماعة العدل والاحسان، أن السلطة المغربية تقوم بـ”استخفاف نخبة عريضة من المجتمعة متمثلة في طلبة كليات الطب والصيدلة وعائلتهم وأساتذتهم من خلال اتهام العدل والاحسان بالوقوف وراء الاحتجاجات”. وقال بيان صادر عن مجلس إرشاد جماعة العدل والاحسان، إن “السلطة المغربية خرجت مرة أخرى، وبطريقة تحريضية مثيرة لكثير من الاستغراب، بأسطوانتها المشروخة، كما فعلت في أحداث سابقة منها أحداث الريف وجرادة، لتتهم العدل والإحسان بالوقوف وراء احتجاجات الطلبة الأطباء”. وتابع البيان تعليقا على بلاغ الحكومة بالقول :”وهو ما يمثل استخفافا بنخبة عريضة من المجتمع متمثلة في طلبة كليات الطب والصيدلة وعائلاتهم وأساتذتهم، واستخفافا بعقول الناس وبوعيهم ، وتعبيرا عن فشلها المتوالي في حل المشاكل المتراكمة للبلد في كل القطاعات، ومحاولة للهروب من تحمل المسؤولية في إيجاد حلول منصفة للمشكل ذي الطبيعة المهنية والأكاديمية الصرفة، ومحاولة يائسة للتغطية على المواجهة المكشوفة التي أصبحت فيها السلطات السياسية والا

100%|██████████| 50/50 [01:17<00:00,  1.55s/it]

****
Actual Article:  يبدو أن ظاهرة الانتحار فمدينة شفشاون معندهاش حد. البارح الأحد عاوتاني لقاو جثة شاب ديال شاب في الثلاثينات من عمره مشنوق بمنزل بدوار تنغاية التابع ترابيا للجماعة القروية باب تازة. حسب ما كشفت عنه مصادر محلية لـ”كود”. وقالت المصادر أن النيابة العامة أمرت بإجراء عملية التشريح الطبي على جثة الهالك الذي كان خدام قيد حياته شيفور، فيما باشرته مصالح الدرك الملكي تحريات وأبحاث لتحديد ملابسات الحادث الغامض. 
Generated Headline:  الانتحار مازال مستمر فشفشاون.. شاب شنق راسو فدوار تنغاية والنيابة العامة أمرت بالتشريح

Average ROUGE-1: 0.0867
Average ROUGE-2: 0.0200
Average ROUGE-L: 0.0867
